In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute, transpile
import tensorflow as tf
import numpy as np
from qiskit.circuit import Parameter, ParameterVector
import qiskit
from qiskit import quantum_info
from qiskit.execute_function import execute
from qiskit import BasicAer

seed = 10599

sys.path.append('../')

In [2]:
from part2_rodrigo import VQC

In [223]:
images=np.load('../data/images.npy')
labels=np.load('../data/labels.npy')

In [4]:
images_resized = np.resize(images, (2000,4,4))

In [6]:
max_images = np.max(np.abs(images_resized))
images_resized_normalized = images_resized/max_images

In [166]:
def encoder_v2(image,threshold):
    info_image = np.ndarray.flatten(image)
    n_qubits = len(info_image)
    q_register = QuantumRegister(n_qubits)
    qc = QuantumCircuit(q_register, name='Encoder')
    for i, pixel_value in enumerate(info_image):
        if pixel_value>=threshold:
            qc.x( q_register[i])
    return qc

In [186]:
n_layers = 1
n_qubits = 16
p = ParameterVector('$\theta$',3*n_layers*n_qubits)
my_vqc = VQC(n_layers,n_qubits,p)
vqc_circuit = my_vqc.vqc()

In [10]:
def execute_circuit(qc, shots=1024, device=None):
    if device is None:
        device = Aer.get_backend('qasm_simulator')
    else:
        device = device
    
    circ_trans = transpile(qc,device)
    counts = device.run(circ_trans, shots=shots).result().get_counts()
    
    return counts

In [16]:
def simulate(circuit: qiskit.QuantumCircuit) -> dict:
    """Simulate the circuit, give the state vector as the result."""
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circuit, backend)
    result = job.result()
    state_vector = result.get_statevector()
    
    histogram = dict()
    for i in range(len(state_vector)):
        population = abs(state_vector[i]) ** 2
        if population > 1e-9:
            histogram[i] = population
    
    return histogram

In [191]:
def histogram_to_category(histogram):
    """This function take a histogram representations of circuit execution results, and process into labels as described in 
    the problem description."""
    assert abs(sum(histogram.values())-1)<1e-8
    positive=0
    for key in histogram.keys():
        digits = bin(int(key))[2:].zfill(20)
        if digits[-1]=='0':
            positive+=histogram[key]
    return positive

In [192]:
from part1 import encoder

In [217]:
def run_part2(image):

    #load the quantum classifier circuit
    classifier=vqc_circuit
    
    #encode image into circuit
    circuit=encoder(image)
    
    #append with classifier circuit
    qc = QuantumCircuit(16,16)
    qc.compose(circuit)
    qc.compose(classifier)
    
    #simulate circuit
    histogram=simulate(qc)
        
    #convert histogram to category
    label=histogram_to_category(histogram)
        
    return circuit,label

In [218]:
from sklearn.metrics import mean_squared_error

In [219]:
def loss(image,label,parameters):
    circuit=encoder(image)
    
    #append with classifier circuit
    qc = QuantumCircuit(16,16)
    qc.compose(circuit)
    qc.compose(vqc_circuit)
    
    #simulate circuit
    histogram=simulate(qc)
        
    #convert histogram to category
    predict = histogram_to_category(histogram)
    #print("Prediction: ", predict)
    #print("Label: ", label)
    return (label-predict)**2

In [220]:
def cost_function(images,labels,parameters):
    cost = []
    labels = [1.0 if labels[i] == True else 0.0 for i in range(len(labels))]
    for i in range(len(images)):
        cost.append(loss(images[i],labels[i],parameters))
    return np.mean(cost)

In [224]:
from sklearn.utils import shuffle
images, labels = shuffle(images, labels)

In [225]:
p = np.random.random(3*n_qubits*n_layers)
dataset = images[0:10]
correct_labels = labels[0:10]
c = cost_function(dataset,correct_labels,p)
print("cost for theta={} ----> {}".format(p,c))

cost for theta=[0.1776316  0.51501418 0.46746423 0.70512694 0.70536217 0.88028533
 0.57288305 0.0892824  0.92320227 0.28973265 0.93172451 0.10692673
 0.99270585 0.84773591 0.4712631  0.69230447 0.4127766  0.31565811
 0.70822682 0.60018257 0.95855525 0.35587159 0.71789555 0.89542806
 0.98674282 0.83397883 0.10703645 0.05247148 0.99702521 0.07420303
 0.16643652 0.3116264  0.52050495 0.11864583 0.48824157 0.6800996
 0.97008811 0.10586447 0.72432271 0.74505282 0.15516682 0.33511945
 0.1658339  0.0497657  0.99184956 0.4255567  0.4564477  0.97702515] ----> 0.7


In [103]:
from qiskit.algorithms.optimizers import COBYLA, ADAM, SPSA #...

# Callback function for optimiser for plotting purposes
def store_intermediate_result(evaluation, parameter, cost, 
                              stepsize, accept):
    evaluations.append(evaluation)
    parameters.append(parameter)
    costs.append(cost)

parameters = []
costs = []
evaluations = []

optimizer = SPSA(maxiter=50,callback=store_intermediate_result)

p = np.random.random(3*n_qubits*n_layers)

objective_function = lambda p: cost_function(images[0:50],labels[0:50],p)
                                              
ret = optimizer.optimize(num_vars=3*n_qubits*n_layers, objective_function=objective_function, initial_point=p)

print("OPTIMIZATION COMPLETED! RESULT ---> {}".format(ret))

C:\Users\rodri\AppData\Local\Temp\ipykernel_24152\148980149.py:20: DeprecationWarning: The SPSA.optimize method is deprecated as of Qiskit Terra 0.21.0 and will be removed no sooner than 3 months after the release date. Instead, use SPSA.minimize as a replacement, which supports the same arguments but follows the interface of scipy.optimize and returns a complete result object containing additional information.
  ret = optimizer.optimize(num_vars=3*n_qubits*n_layers, objective_function=objective_function, initial_point=p)
c:\Users\rodri\AppData\Local\Programs\Python\Python310\lib\site-packages\qiskit\algorithms\optimizers\spsa.py:345: RuntimeWarning: divide by zero encountered in double_scalars
  a = target_magnitude / avg_magnitudes
c:\Users\rodri\AppData\Local\Programs\Python\Python310\lib\site-packages\qiskit\algorithms\optimizers\spsa.py:565: RuntimeWarning: invalid value encountered in multiply
  update = update * next(eta)


OPTIMIZATION COMPLETED! RESULT ---> (array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan]), 0.84, 150)
